# Demo 2: Visualizaciones Avanzadas en E-commerce

En esta demostración, aprenderemos a crear visualizaciones más avanzadas y análisis interactivos para datos de e-commerce.

In [2]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Configurar visualización
plt.style.use('Solarize_Light2')
%matplotlib inline
sns.set_palette('husl')

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 1. Generación de Datos Enriquecidos

In [ ]:
# Generar datos más detallados
np.random.seed(42)

# Crear fechas para un año
fechas = pd.date_range('2024-01-01', '2024-12-31', freq='D')
n_dias = len(fechas)

# Productos y sus características
productos = {
    'Laptop Pro': {'categoria': 'Electrónica', 'precio_base': 1200},
    'Smartphone X': {'categoria': 'Electrónica', 'precio_base': 800},
    'Tablet Air': {'categoria': 'Electrónica', 'precio_base': 500},
    'Smartwatch Elite': {'categoria': 'Accesorios', 'precio_base': 300},
    'Auriculares Pro': {'categoria': 'Accesorios', 'precio_base': 200},
    'Cargador Rápido': {'categoria': 'Accesorios', 'precio_base': 50}
}

# Generar datos de ventas
data = []
for fecha in fechas:
    # Simular más ventas en fines de semana
    n_ventas = np.random.randint(5, 15) if fecha.weekday() >= 5 else np.random.randint(3, 10)
    
    for _ in range(n_ventas):
        producto = np.random.choice(list(productos.keys()))
        precio_base = productos[producto]['precio_base']
        # Agregar variación al precio (+-10%)
        precio = precio_base * np.random.uniform(0.9, 1.1)
        
        data.append({
            'fecha': fecha,
            'producto': producto,
            'categoria': productos[producto]['categoria'],
            'precio': round(precio, 2),
            'cantidad': np.random.randint(1, 4),
            'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste']),
            'canal': np.random.choice(['Online', 'Tienda', 'Distribuidor']),
            'satisfaccion': np.random.randint(1, 6)
        })

# Crear DataFrame
df = pd.DataFrame(data)

# Agregar columnas derivadas
df['ventas'] = df['precio'] * df['cantidad']
df['mes'] = df['fecha'].dt.month
df['dia_semana'] = df['fecha'].dt.day_name()
df['semana'] = df['fecha'].dt.isocalendar().week

print('Dataset generado con éxito. Dimensiones:', df.shape)
df.head()

## 2. Análisis de Tendencias Temporales

In [ ]:
# Crear visualizaciones de tendencias
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# 1. Tendencia de ventas mensuales por categoría
ventas_mensuales = df.groupby(['mes', 'categoria'])['ventas'].sum().unstack()
ventas_mensuales.plot(kind='line', marker='o', ax=axes[0])
axes[0].set_title('Tendencia de Ventas Mensuales por Categoría')
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Ventas ($)')
axes[0].legend(title='Categoría')
axes[0].grid(True)

# 2. Distribución de ventas por día de la semana
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.boxplot(data=df, x='dia_semana', y='ventas', order=order, ax=axes[1])
axes[1].set_title('Distribución de Ventas por Día de la Semana')
axes[1].set_xlabel('Día')
axes[1].set_ylabel('Ventas ($)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Estadísticas por día de la semana
stats_dia = df.groupby('dia_semana')['ventas'].agg(['mean', 'count', 'sum']).round(2)
stats_dia.index = pd.CategoricalIndex(stats_dia.index, categories=order)
stats_dia = stats_dia.sort_index()
display(stats_dia)

## 3. Análisis de Productos y Canales

In [ ]:
# Crear matriz de ventas por producto y canal
matriz_ventas = pd.pivot_table(
    df,
    values='ventas',
    index='producto',
    columns='canal',
    aggfunc='sum',
    margins=True
).round(2)

# Calcular porcentajes por canal
matriz_pct = matriz_ventas.div(matriz_ventas['All'], axis=0) * 100

# Visualizar distribución
plt.figure(figsize=(12, 6))
sns.heatmap(matriz_pct.iloc[:-1, :-1], annot=True, fmt='.1f', cmap='YlOrRd')
plt.title('Distribución de Ventas por Producto y Canal (%)')
plt.xlabel('Canal')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

# Mostrar matriz de ventas
print('\nMatriz de Ventas por Producto y Canal:')
display(matriz_ventas)

## 4. Análisis de Satisfacción del Cliente

In [ ]:
# Crear visualizaciones de satisfacción
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribución general de satisfacción
sns.countplot(data=df, x='satisfaccion', ax=axes[0,0])
axes[0,0].set_title('Distribución de Niveles de Satisfacción')
axes[0,0].set_xlabel('Nivel de Satisfacción')
axes[0,0].set_ylabel('Número de Clientes')

# 2. Satisfacción promedio por producto
df.groupby('producto')['satisfaccion'].mean().sort_values().plot(kind='barh', ax=axes[0,1])
axes[0,1].set_title('Satisfacción Promedio por Producto')
axes[0,1].set_xlabel('Satisfacción Promedio')

# 3. Relación entre precio y satisfacción
sns.boxplot(data=df, x='satisfaccion', y='precio', ax=axes[1,0])
axes[1,0].set_title('Relación entre Precio y Satisfacción')
axes[1,0].set_xlabel('Nivel de Satisfacción')
axes[1,0].set_ylabel('Precio ($)')

# 4. Satisfacción por canal
sns.violinplot(data=df, x='canal', y='satisfaccion', ax=axes[1,1])
axes[1,1].set_title('Distribución de Satisfacción por Canal')
axes[1,1].set_xlabel('Canal')
axes[1,1].set_ylabel('Nivel de Satisfacción')

plt.tight_layout()
plt.show()

# Estadísticas de satisfacción
stats_satisfaccion = df.groupby('producto').agg({
    'satisfaccion': ['mean', 'count', 'std'],
    'ventas': 'sum'
}).round(2)

stats_satisfaccion.columns = ['Satisfacción Media', 'Número de Reviews', 'Desv. Estándar', 'Ventas Totales']
display(stats_satisfaccion.sort_values('Satisfacción Media', ascending=False))

## 5. Análisis de Correlaciones y Patrones

In [ ]:
# Seleccionar variables numéricas
vars_numericas = ['precio', 'cantidad', 'ventas', 'satisfaccion']

# Calcular correlaciones
correlaciones = df[vars_numericas].corr()

# Visualizar matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()

# Análisis de tendencias por categoría y canal
tendencias = df.groupby(['categoria', 'canal']).agg({
    'ventas': ['mean', 'sum', 'count'],
    'satisfaccion': 'mean'
}).round(2)

tendencias.columns = ['Venta Promedio', 'Ventas Totales', 'Número de Ventas', 'Satisfacción Media']
display(tendencias)